# Experiment 7: First Naive Classifier
In this experiment we will be using the very simple processing pipeline from [commit #7](https://github.com/BIAPT/eeg-pain-detection/commits/master) to put together all the plumbing necessary to go from raw data to a trained model.

The model will only be trained on one feature: **Average Alpha Power Across All Channels**
This feature was choosen out of analysis convenience, a better feature would have been power at difference hemisphere however I would have had to deal with missing channels and such. 

The code that will be presented below is very easy to augment by feeding more feature in the data.csv file made by the `preprocessing_pipeline.m` file.

The classifier that will be used here is the very simple [linear support vector machine from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html).

The cross validation scheme we will be using is a leave one subject out cross validation. We'll assess the classifier performance using accuracy (not ideal, but will do for now). 

In [25]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Setup
input_filename = '/media/yacine/Data/pain_and_eeg/machine_learning_data/data.csv'

# Experimental Variables
is_healthy = 1
clf = SVC(gamma='auto')

# Load the data and select the right population
df = pd.read_csv(input_filename) 
df = df[df.type == is_healthy]


p_ids = df.id.unique();
accuracies = []

for p_id in p_ids:
    mask = df.id.isin([p_id])
    
    df_test = df[mask]
    df_train = df[~mask]
    
    
    # Here we are using reshape because we have only one feature and the fit function is looking for 2d array
    X_train = df_train.avg_alpha_power.to_numpy().reshape(-1,1)
    y_train = df_train.is_hot
    X_test = df_test.avg_alpha_power.to_numpy().reshape(-1,1)
    y_test = df_test.is_hot
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
print(np.mean(accuracies))    

0.54041248606466
